# 04: Returns Distribution (リターン分布の基礎)

このノートでは、価格とリターンの違い、単純リターンと対数リターン、分布の歪度・尖度を可視化して学びます。

数式:

$$r_t = \log\left(\frac{P_t}{P_{t-1}}\right)$$

In [ ]:
# --- Imports and configuration ---
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf

SYMBOL = os.getenv("SYMBOL", "QQQ")
PERIOD = "5y"
INTERVAL = "1d"

plt.style.use("seaborn-v0_8")
print(f"SYMBOL={SYMBOL}, PERIOD={PERIOD}, INTERVAL={INTERVAL}")

In [ ]:
# --- Load data and compute returns ---
df = yf.download(SYMBOL, period=PERIOD, interval=INTERVAL, auto_adjust=True, progress=False)
if isinstance(df.columns, pd.MultiIndex):
    close = df[("Close", SYMBOL)].rename("close")
else:
    close = df["Close"].rename("close")

data = pd.DataFrame({"close": close}).dropna()
data["simple_return"] = data["close"].pct_change()
data["log_return"] = np.log(data["close"] / data["close"].shift(1))
data = data.dropna()

data.head()

## 統計量の計算（平均・標準偏差・歪度・尖度）

`scipy` がなくても動くように、手計算関数を用意します。

In [ ]:
# --- Manual skew/kurtosis (scipy-free) ---
def manual_skew(x: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.nan
    m = x.mean()
    s = x.std(ddof=0)
    if s == 0:
        return 0.0
    return np.mean(((x - m) / s) ** 3)

def manual_kurtosis(x: np.ndarray) -> float:
    x = np.asarray(x, dtype=float)
    x = x[np.isfinite(x)]
    if x.size == 0:
        return np.nan
    m = x.mean()
    s = x.std(ddof=0)
    if s == 0:
        return 0.0
    # Pearson kurtosis (normal=3)
    return np.mean(((x - m) / s) ** 4)

for col in ["simple_return", "log_return"]:
    arr = data[col].to_numpy()
    print(f"\n[{col}]")
    print(f"mean     = {arr.mean(): .6f}")
    print(f"std      = {arr.std(ddof=0): .6f}")
    print(f"skew     = {manual_skew(arr): .6f}")
    print(f"kurtosis = {manual_kurtosis(arr): .6f}")

## 可視化: 価格・リターン・ヒストグラム・Quantile Plot

ここでは QQ plot の代替として、
正規分布の理論分位点（モンテカルロ近似）と標本分位点を比較する簡易 Quantile Plot を描きます。

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 9))

# 1) Price
axes[0, 0].plot(data.index, data["close"], color="tab:blue")
axes[0, 0].set_title(f"{SYMBOL} Price")
axes[0, 0].set_ylabel("Price")

# 2) Returns
axes[0, 1].plot(data.index, data["simple_return"], label="Simple Return", alpha=0.7)
axes[0, 1].plot(data.index, data["log_return"], label="Log Return", alpha=0.7)
axes[0, 1].set_title("Simple vs Log Returns")
axes[0, 1].legend()

# 3) Histogram
axes[1, 0].hist(data["log_return"], bins=60, alpha=0.8, color="tab:green")
axes[1, 0].set_title("Histogram of Log Returns")
axes[1, 0].set_xlabel("log return")

# 4) QQ-style quantile plot (without scipy)
rng = np.random.default_rng(42)
x = data["log_return"].to_numpy()
x = x[np.isfinite(x)]
mu, sigma = x.mean(), x.std(ddof=0)
normal_sample = rng.normal(mu, sigma if sigma > 0 else 1e-12, size=200000)
qs = np.linspace(0.01, 0.99, 99)
q_emp = np.quantile(x, qs)
q_theory = np.quantile(normal_sample, qs)

axes[1, 1].scatter(q_theory, q_emp, s=12, alpha=0.7)
qmin, qmax = min(q_theory.min(), q_emp.min()), max(q_theory.max(), q_emp.max())
axes[1, 1].plot([qmin, qmax], [qmin, qmax], color="red", linestyle="--")
axes[1, 1].set_title("QQ-style Quantile Plot")
axes[1, 1].set_xlabel("Normal Quantiles (simulated)")
axes[1, 1].set_ylabel("Empirical Quantiles")

plt.tight_layout()
plt.show()

## What you should notice

- 単純リターンと対数リターンは、日次の小さな変化ではかなり似ます。
- ヒストグラムは正規分布より中心が尖り、裾が厚く見えることがあります。
- Quantile Plot で対角線から乖離するほど、正規仮定との差が大きいと解釈できます。
- これらの特徴は、リスク管理や異常値への頑健性に直結します。